<html>
   <head>
      <title>Dataframe list</title>
   </head>
   <body>
      <h1><font size="5">Part 2: Specialized Dataframes</font></h1>
       <p><font size="4">Dataframes for:</font></p>
      <ul style="list-style-type:none;line-height: 2;">
         <li><font size="3"><a href='#months'>1. Months</a></font></li>
         <ul style="margin-left: 1em; padding-left: 1em;list-style-type:none">
         <li><font size="3"><a href='#retention'>1.1 Retention Rate and Table</a></font></li>
         </ul>          
         <li><font size="3"><a href='#users'>2. Users</a></font></li>
         <li><font size="3"><a href='#articles'>3. Articles</a></font></li>
         <li><font size="3"><a href='#germany'>4. Germany</a></font></li>
      </ul>
   </body>
</html>

In [1]:
# Importing the necessary libraries
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import glob
import datetime
from operator import itemgetter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)

In [3]:
df = pd.read_pickle("./Data/dataframefull.pickle")

<a id='months'></a>

## 1. Dataframe for Months

In [4]:
#total revenue per month
df_month = df.groupby(['year_month'])['total_price'].sum().reset_index()

In [5]:
#women articles per month:
df_month = pd.merge(df_month, df.groupby(['year_month'])['women_article'].sum().reset_index(), on="year_month")
#men articles per month:
df_month = pd.merge(df_month, df.groupby(['year_month'])['men_article'].sum().reset_index(), on="year_month")
#kids articles per month:
df_month = pd.merge(df_month, df.groupby(['year_month'])['kids_article'].sum().reset_index(), on="year_month")
#home and living articles per month:
df_month = pd.merge(df_month, df.groupby(['year_month'])['homeandliving_article'].sum().reset_index(), on="year_month")
#average shopping cart value per month:
df_month = pd.merge(df_month, df.drop_duplicates(subset ="visit_id").groupby(['year_month'])['shop_cart_value'].mean().reset_index(), on="year_month")
#number of active customers per month:
df_month = pd.merge(df_month, df.groupby('year_month')['user_id'].nunique().reset_index(), on="year_month")
#number of total purchases per month:
df_month = pd.merge(df_month, df.groupby('year_month')['visit_id'].nunique().reset_index(), on="year_month")
#average number of items bought per shopping cart:
df_month = pd.merge(df_month, df.drop_duplicates(subset ="visit_id").groupby(['year_month'])['shop_cart_item_count'].mean().reset_index(), on="year_month")


#rename columns to better reflect what they represent
df_month = df_month.rename(columns = {"total_price": "revenue", "shop_cart_value": "avg_shop_cart", "visit_id": "total_purchases", "user_id": "active_customers", "shop_cart_item_count": "avg_cart_itemcount"})

In [6]:
#add monthly growth of revenue:
df_month["monthly_growth_revenue"] = df_month["revenue"].pct_change()
#fill the one NaN that comes with the above code:
df_month = df_month.fillna(0)

In [7]:
#reordering the columns
cols = df_month.columns.tolist()
cols = cols[0:2] + [cols[10]] + [cols[6]] + [cols[9]] + cols[7:9] + cols[2:6] 
print("Check if any column was skipped when reordering (True: all columns fetched):", len(cols) == len(df_month.columns))
df_month = df_month.reindex(columns = cols)

Check if any column was skipped when reordering (True: all columns fetched): True


In [8]:
#revenue for first users
df_month = pd.merge(df_month, df[df["user_type"] == "first"].groupby(['year_month'])["total_price"].sum().reset_index(), on="year_month")
df_month = df_month.rename(columns = {"total_price": "revenue_first"})

In [9]:
#revenue for established users
row = pd.Series({201711: 0},name=0)
est_df = df[df["user_type"] == "established"].groupby(['year_month'])["total_price"].sum().append(row)
est_df = est_df.sort_index()
est_df = est_df.reset_index()
est_df = est_df.rename(columns = {"index": "year_month", 0: "revenue_established"})
df_month = pd.merge(df_month, est_df, on="year_month")

In [10]:
#count of unique first users each month
df_month = pd.merge(df_month, df[df["user_type"] == "first"].groupby(['year_month'])["user_id"].nunique().reset_index(), on="year_month")
df_month = df_month.rename(columns = {"user_id": "first_user_count"})

In [11]:
#count for established users
row = pd.Series({201711: 0},name=0)
estc_df = df[df["user_type"] == "established"].groupby(['year_month'])["user_id"].nunique().append(row)
estc_df = estc_df.sort_index()
estc_df = estc_df.reset_index()
estc_df = estc_df.rename(columns = {"index": "year_month", 0: "established_user_count"})
df_month = pd.merge(df_month, estc_df, on="year_month")

In [12]:
#revenue per month
u_purch = df.groupby(['user_id','year_month'])['total_price'].sum().reset_index()

#retention matrix with crosstab
retention = pd.crosstab(u_purch['user_id'], u_purch['year_month']).reset_index()

#retained and user per month
months = retention.columns[:]
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['year_month'] = int(selected_month)
    retention_data['user_count'] = retention[selected_month].sum()
    retention_data['retained_user_count'] = retention[(retention[selected_month]>0) & (retention[prev_month]>0)][selected_month].sum()
    retention_array.append(retention_data)
    
#array to dataframe and add retention_rate
retention = pd.DataFrame(retention_array)
retention['retention_rate'] = retention['retained_user_count']/retention['user_count']


In [13]:
df_month = pd.merge(df_month, retention.drop(columns = "user_count"), on="year_month")

In [14]:
user_ratio = df.query("user_type == 'first'").groupby(['year_month'])['user_id'].nunique()/df.query("user_type == 'established'").groupby(['year_month'])['user_id'].nunique() 
user_ratio = user_ratio.reset_index()
user_ratio = user_ratio.fillna(0)
user_ratio = user_ratio.rename(columns={"user_id": "ratio_first_to_established"})

In [15]:
df_month = pd.merge(df_month, user_ratio, on="year_month")

In [16]:
df_month.head()

,year_month,revenue,monthly_growth_revenue,avg_shop_cart,avg_cart_itemcount,active_customers,total_purchases,women_article,men_article,kids_article,homeandliving_article,revenue_first,revenue_established,first_user_count,established_user_count,retained_user_count,retention_rate,ratio_first_to_established
0,201711,5.268517e+06,0.000000,103.495016,3.466743,46985,50906,96910,61491,14107,999,5.268517e+06,0.00,46985,0,46985,1.000000,0.000000
1,201712,4.983451e+06,-0.054108,90.652698,3.284649,50004,54973,95202,64962,15461,1806,4.619641e+06,363809.47,46799,3205,3205,0.064095,14.601872
2,201801,3.717325e+06,-0.254066,87.034361,3.661563,38221,42711,89365,52416,10983,1100,3.216473e+06,500851.97,33596,4625,3141,0.082180,7.264000
3,201802,3.056483e+06,-0.177773,90.680675,3.791995,30837,33706,67535,42438,14910,795,2.526812e+06,529670.62,26114,4723,2520,0.081720,5.529113
4,201803,6.210612e+06,1.031947,108.247849,3.966274,52209,57374,139458,66852,18202,964,5.122223e+06,1088389.01,44512,7697,2797,0.053573,5.783032


In [17]:
df_month.to_pickle('Data/months.pickle')

<a id='retention'></a>

### Retention rate shown below:

**retention rate table** from https://towardsdatascience.com/data-driven-growth-with-python-part-1-know-your-metrics-812781e66a5b

In [18]:
#revenue per month
u_purch = df.groupby(['user_id','year_month'])['total_price'].sum().reset_index()

#retention matrix with crosstab
retention = pd.crosstab(u_purch['user_id'], u_purch['year_month']).reset_index()

#retained and uder per month
months = retention.columns[2:]
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['year_month'] = int(selected_month)
    retention_data['user_count'] = retention[selected_month].sum()
    retention_data['retained_user_count'] = retention[(retention[selected_month]>0) & (retention[prev_month]>0)][selected_month].sum()
    retention_array.append(retention_data)
    
#array to dataframe and add retention_rate
retention = pd.DataFrame(retention_array)
retention['retention_rate'] = retention['retained_user_count']/retention['user_count']


In [19]:

#create our retentiond table again with crosstab() - we need to change the column names for using them in .query() function
retentiondd = pd.crosstab(u_purch['user_id'], u_purch['year_month']).reset_index()
new_column_names = [ 'm_' + str(column) for column in retentiondd.columns]
retentiondd.columns = new_column_names

#create the array of Retained users for each cohort monthly
retentiond_array = []
for i in range(len(months)):
    retentiond_data = {}
    selected_month = months[i]
    prev_months = months[:i]
    next_months = months[i+1:]
    for prev_month in prev_months:
        retentiond_data[prev_month] = np.nan
        
    total_user_count =  retentiond_data['total_user_count'] = retentiondd['m_' + str(selected_month)].sum()
    retentiond_data[selected_month] = 1 
    
    plch = "{} > 0".format('m_' + str(selected_month))
    

    for next_month in next_months:
        plch = plch + " and {} > 0".format(str('m_' + str(next_month)))
        retentiond_data[next_month] = np.round(retentiondd.query(plch)['m_' + str(next_month)].sum()/total_user_count,4)
    retentiond_array.append(retentiond_data)
    
retentiondd = pd.DataFrame(retentiond_array)
retentiondd.index = months

#showing new cohort based retentiond table
retentiondd



,total_user_count,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902
year_month,,,,,,,,,,,,,,,,
201712,50004,1.0,0.0628,0.0128,0.0057,0.0026,0.0016,0.0011,0.0007,0.0005,0.0004,0.0003,0.0003,0.0003,0.0002,0.0002
201801,38221,NaN,1.0000,0.0659,0.0182,0.0062,0.0036,0.0023,0.0015,0.0010,0.0007,0.0006,0.0005,0.0005,0.0004,0.0003
201802,30837,NaN,NaN,1.0000,0.0907,0.0215,0.0103,0.0051,0.0031,0.0018,0.0014,0.0010,0.0008,0.0007,0.0006,0.0005
201803,52209,NaN,NaN,NaN,1.0000,0.0727,0.0201,0.0076,0.0038,0.0021,0.0014,0.0010,0.0008,0.0007,0.0004,0.0003
201804,44879,NaN,NaN,NaN,NaN,1.0000,0.0887,0.0199,0.0085,0.0041,0.0025,0.0016,0.0012,0.0010,0.0006,0.0004
201805,54426,NaN,NaN,NaN,NaN,NaN,1.0000,0.0774,0.0215,0.0077,0.0038,0.0024,0.0017,0.0012,0.0008,0.0006
201806,45145,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,0.0931,0.0220,0.0079,0.0047,0.0029,0.0019,0.0013,0.0010
201807,48122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,0.0794,0.0165,0.0075,0.0041,0.0026,0.0017,0.0012
201808,38260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,0.0781,0.0231,0.0101,0.0053,0.0032,0.0021


Sadly this shows that there is close to zero retention happening in the webshopdata. Thus further analysis of retention rate and, also in respect to this, customer churning will not yield any satisfying results.


<a id='users'></a>

## 2. Dataframe for Users

In [20]:
#initialize dataframe with total revenue per customer
df_user = df.groupby('user_id')['total_price'].sum().reset_index()

In [21]:
#women articles the user bought:
df_user = pd.merge(df_user, df.groupby(['user_id'])['women_article'].sum().reset_index(), on="user_id")
#men articles the user bought:
df_user = pd.merge(df_user, df.groupby(['user_id'])['men_article'].sum().reset_index(), on="user_id")
#kids articles the user bought:
df_user = pd.merge(df_user, df.groupby(['user_id'])['kids_article'].sum().reset_index(), on="user_id")
#home and living articles the user bought:
df_user = pd.merge(df_user, df.groupby(['user_id'])['homeandliving_article'].sum().reset_index(), on="user_id")
#average shopping cart value per customer:
df_user = pd.merge(df_user, df.drop_duplicates(subset ="visit_id").groupby(['user_id'])['shop_cart_value'].mean().reset_index(), on="user_id")
#number of total visits per user:
df_user = pd.merge(df_user, df.groupby('user_id')['visit_id'].nunique().reset_index(), on="user_id")
#average number of items bought per shopping cart:
df_user = pd.merge(df_user, df.drop_duplicates(subset ="visit_id").groupby(['user_id'])['shop_cart_item_count'].mean().reset_index(), on="user_id")


#rename columns to better reflect what they represent
df_user = df_user.rename(columns = {"total_price": "revenue", "shop_cart_value": "avg_shop_cart", "visit_id": "total_purchases", "shop_cart_item_count": "avg_cart_itemcount"})

In [22]:
# entry for date + user for easier use
df["date_user"] = df['date'].map(lambda date: 10000*date.year + 100*date.month + date.day).apply(lambda x: str(x)) + df.user_id.apply(lambda x: str(x))

In [23]:
df.day_of_week.replace(0, 7, inplace=True)

In [24]:
def setday(day, daynr):
    """
    Set the days for users in the df_user dataframe
    day - weekday , Monday .... Sunday
    daynr - nr of day , here 7,1,2,...,6
    """
    uniq_d = df[df["day_of_week"] == daynr].drop_duplicates(subset = "date_user").groupby('user_id')["day_of_week"].sum().reset_index()
    uniq_d = uniq_d.set_index("user_id")
    del uniq_d.index.name
    uniq_calc = (uniq_d//daynr)
    uniq_dic = uniq_calc.to_dict()
    num_d = df[df["day_of_week"] == daynr].drop_duplicates(subset ="visit_id").groupby('user_id')["day_of_week"].sum().reset_index()
    num_d = num_d.set_index("user_id")
    num_d_calc = ((num_d - uniq_d)//daynr)+1  #articles per day
    num_dic = num_d_calc.to_dict()
    df_user[str(day)] = df_user["user_id"].map(uniq_dic["day_of_week"])
    df_user["times_bought_" + str(day)] = df_user["user_id"].map(num_dic["day_of_week"])

In [25]:
daylist = [("monday", 7), ("tuesday", 1), ("wednesday", 2), ("thursday", 3), ("friday", 4), ("saturday", 5), ("sunday", 6)]

In [26]:
for i, j in daylist:
    setday(i,j)

In [27]:
df_user = df_user.fillna(0)

In [28]:
#replace monday back to 0
df.day_of_week.replace(7, 0, inplace=True)

#### usertype categories

In [29]:
type_df = df.groupby('user_id')['user_type'].sum().reset_index()

def check_type(item):
    return "first" if re.search(r"established", item) == None else "established"

type_df['user_type'] = type_df['user_type'].apply(check_type)

In [30]:
df_user = pd.merge(df_user, type_df, on="user_id")

In [31]:
df_country = df.groupby('user_id')["country"].apply(lambda x: ', '.join(x))

In [32]:
#get word with the most occurence in a string
def freq(x): 
    out = []
    # break the string into list of words 
    str_list = x.split(sep=", ") 
  
    # gives set of unique words 
    unique_words = set(str_list) 
      
    for words in unique_words : 
        out.append((words, str_list.count(words)))
    return max(out,key=itemgetter(1))[0] 

In [33]:
list_columns = ["region", "city", "device_category", "device_browser", "device_os"]

In [34]:
def stringbuilder(columnlist):
    df_out = df.groupby('user_id')["country"].apply(lambda x: ', '.join(x)).reset_index()
    for j in columnlist:
        df_out = df_out.join(df.groupby('user_id')[j].apply(lambda x: ', '.join(x)).reset_index().drop(columns="user_id"))
    return df_out

In [35]:
df_strings = stringbuilder(list_columns)

In [36]:
#df_strings = pd.read_pickle('Data/datastrings.pickle')

In [37]:
df_strings = df_strings.drop(columns=["user_id"])

In [38]:
for x in df_strings.columns:
    df_strings[x] = df_strings[x].apply(freq)

In [39]:
def slim_country(item):
    if item == "Germany":
        return "Germany"
    elif item == "Austria":
        return "Austria"
    else:
        return "Other"

df_strings['country_slim'] = df_strings['country'].apply(slim_country)


In [40]:
#save the string dataframe as the code took quite long to run, so if the notebook needs to rerun i have it
df_strings.to_pickle('Data/datastrings.pickle')

In [41]:
df_user = df_user.join(df_strings)

In [42]:
df_user.to_pickle('Data/users.pickle')

<a id='articles'></a>

## 3. Dataframe for Articles

In [43]:
art_plchldr = df[["visit_id", "date", "article_id", "article_name", "price",\
                  "women_article", "men_article", "kids_article", "homeandliving_article" \
                 , "category_slim", "brand"]]

In [44]:
df_articles = art_plchldr.sort_values(["article_id", "date"])

In [45]:
df_articles.to_pickle('Data/articles.pickle')

<a id='germany'></a>

## 4. Dataframes for Germany

In [46]:
df_ger = df[df["country"] == "Germany"]

In [47]:
df_ger_sorted = df_ger.sort_values(["region", "city"])

In [48]:
df_ger_states = pd.DataFrame(df_ger_sorted.drop_duplicates(subset ="visit_id").region.value_counts())
df_ger_states.reset_index(inplace=True)
df_ger_states = df_ger_states.rename(columns = {"index": "region", "region": "total_purchases"})

In [49]:
#women articles per region:
df_ger_states = pd.merge(df_ger_states, df_ger_sorted.groupby(['region'])['women_article'].sum().reset_index(), on="region")
#men articles per region:
df_ger_states = pd.merge(df_ger_states, df_ger_sorted.groupby(['region'])['men_article'].sum().reset_index(), on="region")
#kids articles per region:
df_ger_states = pd.merge(df_ger_states, df_ger_sorted.groupby(['region'])['kids_article'].sum().reset_index(), on="region")
#home and living articles per region:
df_ger_states = pd.merge(df_ger_states, df_ger_sorted.groupby(['region'])['homeandliving_article'].sum().reset_index(), on="region")
#average shopping cart value per region:
df_ger_states = pd.merge(df_ger_states, df_ger_sorted.drop_duplicates(subset ="visit_id").groupby(['region'])['shop_cart_value'].mean().reset_index(), on="region")
#number of active customers per region:
df_ger_states = pd.merge(df_ger_states, df_ger_sorted.groupby('region')['user_id'].nunique().reset_index(), on="region")
#average number of items bought per shopping cart:
df_ger_states = pd.merge(df_ger_states, df_ger_sorted.drop_duplicates(subset ="visit_id").groupby(['region'])['shop_cart_item_count'].mean().reset_index(), on="region")


df_ger_states = df_ger_states.rename(columns = {"total_price": "revenue", "shop_cart_value": "avg_shop_cart", "user_id": "active_customers", "shop_cart_item_count": "avg_cart_itemcount"})

In [50]:
def set_casual(region):
    return df_ger_sorted[df_ger_sorted["region"] == str(region)].brand.value_counts().reset_index().brand.loc[0]
def set_denim(region):
    return df_ger_sorted[df_ger_sorted["region"] == str(region)].brand.value_counts().reset_index().brand.loc[1]                  
def set_notset(region):
    return df_ger_sorted[df_ger_sorted["region"] == str(region)].brand.value_counts().reset_index().brand.loc[2]                       

In [51]:
df_ger_states["casual_brand_count"] = df_ger_states["region"].apply(set_casual)
df_ger_states["denim_brand_count"] = df_ger_states["region"].apply(set_denim)
df_ger_states["(not_set)_brand_count"] = df_ger_states["region"].apply(set_notset)

In [52]:
df_ger_states.to_pickle('Data/gerstates.pickle')
df_ger_sorted.to_pickle('Data/gersorted.pickle')